In [1]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [2]:
conda install -c conda-forge keras-rectified-adam


Note: you may need to restart the kernel to use updated packages.


usage: conda-script.py [-h] [-V] command ...
conda-script.py: error: unrecognized arguments: keras-rectified-adam


In [1]:
import logging, os
import random
import tqdm
import keras
import warnings
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras.models import Sequential, Model
from sklearn.model_selection import LeaveOneOut
from keras.layers import Dense, Input, concatenate
from keras.losses import binary_crossentropy
from keras.utils import plot_model
from keras_radam import RAdam
from keras import backend as K

warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'keras_radam'

In [ ]:
#Configurar a semente (seed) para a geração de números pseudoaleatórios
seed_value = 0

#Set `PYTHONHASHSEED` environment variable at a fixed value
logging.disable(logging.WARNING)
os.environ['PYTHONHASHSEED'] = str(seed_value)

#Set `python` built-in pseudo-random generator at a fixed value
random.seed(seed_value)

#Set `numpy` pseudo-random generator at a fixed value
np.random.seed(seed_value)

#Set `tensorflow` pseudo-random generator at a fixed value
tf.compat.v1.set_random_seed(seed_value)

#Configure a new global `tensorflow` session
session_conf = tf.compat.v1.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

In [ ]:
# Definir as características categóricas e seus mapeamentos
str_att = {
  'length': ['short', 'long'],
  'shape': ['closedrect', 'dblopnrect', 'ellipse', 'engine', 'hexagon',
          'jaggedtop', 'openrect', 'opentrap', 'slopetop', 'ushaped'],
  'load_shape': ['circlelod', 'hexagonlod', 'rectanglod', 'trianglod'],
  'Class_attribute': ['west','east']
}

# Ler os dados do arquivo CSV
data = pd.read_csv('dados-trens.csv')

# Transformar os valores categóricos em numéricos
data['Class_attribute'] = data['Class_attribute'].map({'east': 1, 'west': -1})

# Aplicar os mapeamentos para características categóricas
for k in data:
    for att in str_att:
        if k.startswith(att):
            for i, val in enumerate(data[k]):
                if val in str_att[att]:
                    data.at[i, k] = str_att[att].index(val)

# Substituir valores específicos por outros
data.replace("\\0", 0, inplace=True)
data.replace("None", -1, inplace=True)


data = data.fillna(-1)

# Para primeiro teste, recebe um indo para o oeste e dois para o leste e remove dos casos de treinamento.
array_test = []
test_data = random.randint(6,9)
x = random.randint(0, 4)
y = random.randint(0, 4)
while (x == y):
    y = random.randint(0,4)
array_test.append([test_data, x, y])
array_test = array_test[0]

first_training_data = data.drop(array_test)
first_test_data = data.iloc[array_test]

# Separar as características (entradas) e o rótulo (saída) da classe
X = first_training_data.drop('Class_attribute', axis=1)
y = first_training_data['Class_attribute']
X_test = first_test_data.drop('Class_attribute', axis=1)
y_test = first_test_data['Class_attribute']


array_test = []
test_data = random.randint(0, 4)
x = random.randint(5, 9)
y1 = random.randint(5, 9)
while (x == y1):
    y1 = random.randint(5, 9)
array_test.append([test_data, x, y1])
array_test = array_test[0]

scnd_training_data = data.drop(array_test)
scnd_test_data = data.iloc[array_test]

# Separar as características (entradas) e o rótulo (saída) da classe
X2 = scnd_training_data.drop('Class_attribute', axis=1)
y2 = scnd_training_data['Class_attribute']
X_test2 = scnd_test_data.drop('Class_attribute', axis=1)
y_test2 = scnd_test_data['Class_attribute']

# Função para criar a rede neural
def create_neural_network():
    model = Sequential()
    model.add(Dense(units=10, activation='linear', input_shape=(X.shape[1],)))
    model.add(Dense(units=1, activation='tanh'))
    return model

# Questão 1:

In [ ]:
#teste 1: 

model = create_neural_network()

X_train_tensor = tf.convert_to_tensor(X.values, dtype=tf.float32)
y_train_tensor = tf.convert_to_tensor(y.values, dtype=tf.float32)
X_test_tensor = tf.convert_to_tensor(X_test.values, dtype=tf.float32)
y_test_tensor = tf.convert_to_tensor(y_test.values, dtype=tf.float32)

# Compilar o modelo
model.compile(optimizer='adam', loss='mse')

# Treinar a rede neural
model.fit(X_train_tensor, y_train_tensor, epochs=20, verbose=0)
predictions = model.predict(X_test_tensor)

predictions[predictions >= 0] = 1.0
predictions[predictions < 0] = -1.0

print(y_test_tensor)

counter = 0
for i in range(0, 3):
    print(predictions[i][0], " -> ", y_test_tensor.numpy().tolist()[i])
    if predictions[i][0] == y_test_tensor.numpy().tolist()[i]:
        counter += 1

print("Acurácia: ", (counter * 100 / 3), "%")

In [ ]:
#teste2

model = create_neural_network()

X_train_tensor = tf.convert_to_tensor(X2.values, dtype=tf.float32)
y_train_tensor = tf.convert_to_tensor(y2.values, dtype=tf.float32)
X_test_tensor = tf.convert_to_tensor(X_test2.values, dtype=tf.float32)
y_test_tensor = tf.convert_to_tensor(y_test2.values, dtype=tf.float32)

# Compilar o modelo
model.compile(optimizer='adam', loss='mse')

# Treinar a rede neural
model.fit(X_train_tensor, y_train_tensor, epochs=20, verbose=0)
predictions = model.predict(X_test_tensor)

predictions[predictions >= 0] = 1.0
predictions[predictions < 0] = -1.0

counter = 0
for i in range(0, 3):
    print(predictions[i][0], " -> ", y_test_tensor.numpy().tolist()[i])
    if predictions[i][0] == y_test_tensor.numpy().tolist()[i]:
        counter += 1

print("Acurácia: ", (counter * 100 / 3), "%")

# Questão 2:

In [9]:
# path to the data
path = "dados-trens.csv"


def read_data(path=path):
    df = pd.read_csv(path, ",")

    for k in df:
        for att in str_att:
            if k.startswith(att):
                for i, val in enumerate(df[k]):
                    if val in str_att[att]:
                        df[k][i] = str_att[att].index(val)

    df.replace("\\0", 0, inplace=True)
    df.replace("None", -1, inplace=True)

    return df

In [7]:
def model_2():
    # features
    t = Input(shape=(1,), name="t")
    c = Input(shape=(1,), name="c")
    w = Input(shape=(1,), name="w")
    l = Input(shape=(1,), name="l")
    s = Input(shape=(1,), name="s")
    nc = Input(shape=(1,), name="nc")
    ls = Input(shape=(1,), name="ls")
    nl = Input(shape=(1,), name="nl")
    ncl = Input(shape=(1,), name="ncl")
    x_crc = Input(shape=(1,), name="x_crc")
    x_hex = Input(shape=(1,), name="x_hex")
    x_rec = Input(shape=(1,), name="x_rec")
    x_tri = Input(shape=(1,), name="x_tri")

    # num_cars(t,nc)
    num_cars_ = concatenate([t, nc])
    num_cars_ = Dense(20, activation="relu")(num_cars_)
    num_cars = Dense(1, activation="sigmoid", name="num_cars")(num_cars_)
    num_cars = Model(inputs=[t, nc], outputs=num_cars)

    # num_loads(t,nl)
    num_loads_ = concatenate([t, nl])
    num_loads_ = Dense(20, activation="relu")(num_loads_)
    num_loads = Dense(1, activation="sigmoid", name="num_loads")(num_loads_)
    num_loads = Model(inputs=[t, nl], outputs=num_loads)

    # num_wheels(t,c,w)
    num_wheels_ = concatenate([t, c, w])
    num_wheels_ = Dense(20, activation="relu")(num_wheels_)
    num_wheels = Dense(1, activation="sigmoid", name="num_wheels")(num_wheels_)
    num_wheels = Model(inputs=[t, c, w], outputs=num_wheels)

    # length(t,c,l)
    length_ = concatenate([t, c, l])
    length_ = Dense(20, activation="relu")(length_)
    length = Dense(1, activation="sigmoid", name="length")(length_)
    length = Model(inputs=[t, c, l], outputs=length)

    # shape(t,c,s)
    shape_ = concatenate([t, c, s])
    shape_ = Dense(20, activation="relu")(shape_)
    shape = Dense(1, activation="sigmoid", name="shape")(shape_)
    shape = Model(inputs=[t, c, s], outputs=shape)

    # num_car_loads(t,c,ncl)
    num_car_loads_ = concatenate([t, c, ncl])
    num_car_loads_ = Dense(20, activation="relu")(num_car_loads_)
    num_car_loads = Dense(1, activation="sigmoid", name="num_car_loads")(num_car_loads_)
    num_car_loads = Model(inputs=[t, c, ncl], outputs=num_car_loads)

    # load_shape(t,c,ls)
    load_shape_ = concatenate([t, c, ls])
    load_shape_ = Dense(20, activation="relu")(load_shape_)
    load_shape = Dense(1, activation="sigmoid", name="load_shape")(load_shape_)
    load_shape = Model(inputs=[t, c, ls], outputs=load_shape)

    # next_crc(t,c,x)
    next_crc_ = concatenate([t, c, x_crc])
    next_crc_ = Dense(20, activation="relu")(next_crc_)
    next_crc = Dense(1, activation="sigmoid", name="next_crc")(next_crc_)
    next_crc = Model(inputs=[t, c, x_crc], outputs=next_crc)

    # next_hex_(t,c,x)
    next_hex_ = concatenate([t, c, x_hex])
    next_hex_ = Dense(20, activation="relu")(next_hex_)
    next_hex = Dense(1, activation="sigmoid", name="next_hex")(next_hex_)
    next_hex = Model(inputs=[t, c, x_hex], outputs=next_hex)

    # next_rec(t,c,x)
    next_rec_ = concatenate([t, c, x_rec])
    next_rec_ = Dense(20, activation="relu")(next_rec_)
    next_rec = Dense(1, activation="sigmoid", name="next_rec")(next_rec_)
    next_rec = Model(inputs=[t, c, x_rec], outputs=next_rec)

    # next_tri(t,c,x)
    next_tri_ = concatenate([t, c, x_tri])
    next_tri_ = Dense(20, activation="relu")(next_tri_)
    next_tri = Dense(1, activation="sigmoid", name="next_tri")(next_tri_)
    next_tri = Model(inputs=[t, c, x_tri], outputs=next_tri)

    # east
    east = concatenate(
        [
            num_cars_,
            num_loads_,
            num_wheels_,
            length_,
            shape_,
            num_car_loads_,
            load_shape_,
            next_crc_,
            next_hex_,
            next_rec_,
            next_tri_,
        ]
    )
    east = Dense(3, activation="relu")(east)
    east = Dense(1, activation="sigmoid", name="east")(east)
    east = Model(
        inputs=[t, c, w, l, s, nc, ls, nl, ncl, x_crc, x_hex, x_rec, x_tri],
        outputs=east,
    )

    # metanet
    metanet = Model(
        inputs=east.inputs,
        outputs=[
            num_cars.output,
            num_loads.output,
            num_wheels.output,
            length.output,
            shape.output,
            num_car_loads.output,
            load_shape.output,
            next_crc.output,
            next_hex.output,
            next_rec.output,
            next_tri.output,
            east.output,
        ],
    )

    # rules
    rules = {
        "num_cars": num_cars,
        "num_loads": num_loads,
        "num_wheels": num_wheels,
        "length": length,
        "shape": shape,
        "num_car_loads": num_car_loads,
        "load_shape": load_shape,
        "next_crc": next_crc,
        "next_hex": next_hex,
        "next_rec": next_rec,
        "next_tri": next_tri,
    }
    
    # Camada de meta-aprendizado
    meta_input = concatenate([
        num_cars.output,
        num_loads.output,
        num_wheels.output,
        length.output,
        shape.output,
        num_car_loads.output,
        load_shape.output,
        next_crc.output,
        next_hex.output,
        next_rec.output,
        next_tri.output,
        east.output
    ])
    meta_hidden = Dense(20, activation="relu")(meta_input)
    meta_output = Dense(1, activation="sigmoid", name="meta_output")(meta_hidden)
    meta_net = Model(inputs=metanet.inputs, outputs=meta_output)

    # Adicionar meta-aprendizado à saída do modelo leste (east)
    east_output = east.outputs
    east_output.append(meta_output)
    east = Model(inputs=east.inputs, outputs=east_output)

    return metanet, east, rules

In [8]:
# Divisão dos dados em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Compilação do modelo
metanet, east, rules = model_2()
east.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Treinamento do modelo
east.fit(X_train, y_train, epochs=10, batch_size=32)

# Avaliação do modelo
loss, accuracy = east.evaluate(X_test, y_test)
print("Acurácia:", accuracy)

NameError: name 'Input' is not defined